In [1]:
%install-location $cwd/swift-install
%install '.package(path: "$cwd/SchwiftyNotebook_matrix_multiplication")' SchwiftyNotebook_matrix_multiplication

Installing packages:
	.package(path: "/home/ubuntu/workspace/fast-ai-swift/SchwiftyNotebook_matrix_multiplication")
		SchwiftyNotebook_matrix_multiplication
With SwiftPM flags: []
Working in: /tmp/tmp58ahsosi/swift-install
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/home/ubuntu/swift/usr/bin/swift: /home/ubuntu/anaconda3/lib/libuuid.so.1: no version information available (required by /home/ubuntu/swift/usr/bin/swift)
/h

In [2]:
import Path
import TensorFlow
import SchwiftyNotebook_matrix_multiplication

In [3]:
//export
public extension Tensor where Scalar: TensorFlowFloatingPoint {
    init(kaimingNormalInit shape: TensorShape, negativeSlope: Double = 1.0){
        let gain = Scalar.init(TensorFlow.sqrt(2.0 / (1.0 + TensorFlow.pow(negativeSlope, 2))))
        let spatialDimCount = shape.count - 2
        let receptiveField = shape[0..<spatialDimCount].contiguousSize
        let fanIn = shape[spatialDimCount] * receptiveField
        self.init(randomNormal: shape)
        self *= Tensor<Scalar>(gain/TensorFlow.sqrt(Scalar(fanIn)))
    }
}

In [4]:
//export
public protocol SchwiftyLayer: Layer {
    
    var outputCallbacks: [(Output) -> ()] {get set} // allows you to hook into the outputs
    
    @differentiable
    func forwardPass(_ input: Input) -> Output
    
    associatedtype Input
    associatedtype Output
}

In [7]:
//export
public extension SchwiftyLayer {
    
    func callGradient(_ input: Input) -> 
    (Output, (Self.Output.TangentVector) -> (Self.TangentVector, Self.Input.TangentVector)){
        return Swift.valueWithPullback(at: self, input) {
            (m, i) in m(i)
        }
    }
    
    @differentiable(vjp: callGradient)
    func callAsFunction(_ input: Input) -> Output {
        let activations = forwardPass(input)
        for callback in outputCallbacks { callback(activations)}
        return activations
    }
    
    
    var outputCallbacks: [(Output) -> ()]{
        get { return []}
        set {}
    }
    
    mutating func addOutputCallBack(_ hook: @escaping (Output) -> ()) {
        outputCallbacks.append(hook)
    }

}